In [2]:
import numpy as np
import gym
env = gym.make('MountainCar-v0')

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


In [3]:
def semi_gradient_sarsa(env,n_episodes,gamma,alpha,epsilon):
    
    n_actions = env.action_space.n
    
    def epsilon_greedy_policy(state,epsilon,n_actions):
        if np.random.rand() < epsilon:
            return np.random.randint(n_actions)
        else:
            values = []
            for action in range(n_actions):
                values.append(q(state,action,w))
            return np.argmax(values)
    
    def q(s,a,w):
        features = np.append(s,a)
        return np.dot(features,w)
    
    w = np.zeros((3))
    
    for i_episode in range(n_episodes):
        
        #initial state and action
        observation = env.reset()
        action = epsilon_greedy_policy(observation,epsilon,n_actions)
        
        done = False
        
        while not done:
            
            old_observation = observation
            old_action = action
            #take action, observe R, S'
            observation, reward, done, info = env.step(old_action)
            
            if done: #if S' (observation) is terminal
                gradient = np.append(old_observation,old_action)
                w += alpha * (reward + q(old_observation,old_action,w)) * gradient
                break
            
            #choose A' as function of q(S',w)
            action = epsilon_greedy_policy(observation,epsilon,n_actions)
            
            gradient = np.append(old_observation,old_action)
            w += alpha * (reward + gamma * q(observation,action,w) - q(old_observation,old_action,w))*gradient
    return w

In [4]:
w = semi_gradient_sarsa(env,5000,0.9,0.01,0.1)

In [5]:
w

array([ 1.55001250e+01,  1.87471000e+01, -3.50486278e-03])

In [6]:
def greedy_policy(state,n_actions):
    values = []
    for action in range(n_actions):
        values.append(q(state,action,w))
    return np.argmax(values)

def q(s,a,w):
    features = np.append(s,a)
    return np.dot(features,w)

In [9]:
scores = []
choices = []

for i_episode in range(100):
    score = 0
    
    observation = env.reset()
    done = False
    
    while not done:
        env.render()
        old_observation = observation
        action = greedy_policy(old_observation,3)
        observation, reward, done, info = env.step(action)
        score += reward
    scores.append(score)

print("Average score:", sum(scores)/len(scores))

KeyboardInterrupt: 